In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import fugashi

text = """
５．家に帰るとき
大きな地震が起こると、電車やバスなどが止まることがあります。急いで家に帰ろうとしないで、会社や学校など安全な場所でしばらく待ってください。大勢の人が同じ時間に帰ろうとすると、道や駅などが混んで危険だからです。テレビやインターネットなどで調べて、安全だとわかってから帰りましょう。

６．家族や友達に連絡するとき

地震のあとは大勢の人が電話を使うため、家族や友達に連絡しにくくなります。電話番号１７１の「災害用伝言ダイヤル」を使うと、会話はできなくてもメッセージを録音したり、聞いたりすることができます。携帯電話会社の「災害用伝言板」などでもメッセージを送ることができますから、使い方を調べておきましょう。

７．家で生活ができないとき
建物が壊れたりして家で生活ができなくなった場合、市や町などが決めた「避難所」にしばらくいることができます。大勢の人と一緒に生活をしますから、健康に気をつけましょう。

狭い場所で長い時間体を動かさないと、「エコノミークラス症候群」という病気になることもあります。車の中で生活をする人は特に気をつけましょう。
""".strip()

In [23]:
tagger = fugashi.Tagger()

In [24]:
print(tagger(text))

[５, ．, 家, に, 帰る, とき, 大きな, 地震, が, 起こる, と, 、, 電車, や, バス, など, が, 止まる, こと, が, あり, ます, 。, 急い, で, 家, に, 帰ろう, と, し, ない, で, 、, 会社, や, 学校, など, 安全, な, 場所, で, しばらく, 待っ, て, ください, 。, 大勢, の, 人, が, 同じ, 時間, に, 帰ろう, と, する, と, 、, 道, や, 駅, など, が, 混ん, で, 危険, だ, から, です, 。, テレビ, や, インターネット, など, で, 調べ, て, 、, 安全, だ, と, わかっ, て, から, 帰り, ましょう, 。, ６, ．, 家族, や, 友達, に, 連絡, する, とき, 地震, の, あと, は, 大勢, の, 人, が, 電話, を, 使う, ため, 、, 家族, や, 友達, に, 連絡, し, にくく, なり, ます, 。, 電話, 番号, １７１, の, 「, 災害, 用, 伝言, ダイヤル, 」, を, 使う, と, 、, 会話, は, でき, なく, て, も, メッセージ, を, 録音, し, たり, 、, 聞い, たり, する, こと, が, でき, ます, 。, 携帯, 電話, 会社, の, 「, 災害, 用, 伝言, 板, 」, など, で, も, メッセージ, を, 送る, こと, が, でき, ます, から, 、, 使い, 方, を, 調べ, て, おき, ましょう, 。, ７, ．, 家, で, 生活, が, でき, ない, とき, 建物, が, 壊れ, たり, し, て, 家, で, 生活, が, でき, なく, なっ, た, 場合, 、, 市, や, 町, など, が, 決め, た, 「, 避難, 所, 」, に, しばらく, いる, こと, が, でき, ます, 。, 大勢, の, 人, と, 一緒, に, 生活, を, し, ます, から, 、, 健康, に, 気, を, つけ, ましょう, 。, 狭い, 場所, で, 長い, 時間, 体, を, 動かさ, ない, と, 、, 「, エコノミー, クラス, 症候, 群, 」, と, いう, 病気, に, なる, こと, も, あり, ます, 。, 車, の

In [25]:
tokens = tagger(text)
tokens[0]

５

In [26]:
tab = "\t"
for token in tokens:
    print(f"{token}\t{str.join(tab, token.feature_raw.split(','))}")

５	名詞	数詞	*	*	*	*
．	補助記号	句点	*	*	*	*	*	．	．	*	．	*	記号	*	*	*	*	*	*	補助	*	*	*	*	*	*	*	14027396882944	51
家	名詞	普通名詞	一般	*	*	*	イエ	家	家	イエ	家	イエ	和	*	*	*	*	*	*	体	イエ	イエ	イエ	イエ	2	C3	*	434315716600320	1580
に	助詞	格助詞	*	*	*	*	ニ	に	に	ニ	に	ニ	和	*	*	*	*	*	*	格助	ニ	ニ	ニ	ニ	*	名詞%F1	*	7745518285496832	28178
帰る	動詞	一般	*	*	五段-ラ行	連体形-一般	カエル	返る	帰る	カエル	帰る	カエル	和	*	*	*	*	*	*	用	カエル	カエル	カエル	カエル	1	C1	*	1643503629247169	5979
とき	名詞	普通名詞	副詞可能	*	*	*	トキ	時	とき	トキ	とき	トキ	和	ト濁	基本形	*	*	*	*	体	トキ	トキ	トキ	トキ	2	C3	*	7210331000676864	26231
大きな	連体詞	*	*	*	*	*	オオキナ	大きな	大きな	オーキナ	大きな	オーキナ	和	*	*	*	*	*	*	相	オオキナ	オオキナ	オオキナ	オオキナ	1	*	*	1254551424475648	4564
地震	名詞	普通名詞	一般	*	*	*	ジシン	地震	地震	ジシン	地震	ジシン	漢	*	*	*	*	*	*	体	ジシン	ジシン	ジシン	ジシン	0	C2	*	4908503407927808	17857
が	助詞	格助詞	*	*	*	*	ガ	が	が	ガ	が	ガ	和	*	*	*	*	*	*	格助	ガ	ガ	ガ	ガ	*	"動詞%F2@0	名詞%F1"	*	2168520431510016	7889
起こる	動詞	一般	*	*	五段-ラ行	終止形-一般	オコル	起こる	起こる	オコル	起こる	オコル	和	*	*	*	*	*	*	用	オコル	オコル	オコル	オコル	2	C1	*	1341137965163179	4879
と	助詞	接続助詞	*	*	*	*	ト	と	と	ト	と	ト	和	*	*	*	*	*	*	接助	ト	ト	ト	ト	*	"形容詞%F2@-1	動詞%F1"	*	7098730570457600	25825

In [27]:
import pandas as pd
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 50
token.feature._asdict()

{'pos1': '補助記号',
 'pos2': '句点',
 'pos3': '*',
 'pos4': '*',
 'cType': '*',
 'cForm': '*',
 'lForm': '*',
 'lemma': '。',
 'orth': '。',
 'pron': '*',
 'orthBase': '。',
 'pronBase': '*',
 'goshu': '記号',
 'iType': '*',
 'iForm': '*',
 'fType': '*',
 'fForm': '*',
 'iConType': '*',
 'fConType': '*',
 'type': '補助',
 'kana': '*',
 'kanaBase': '*',
 'form': '*',
 'formBase': '*',
 'aType': '*',
 'aConType': '*',
 'aModType': '*',
 'lid': '6880571302400',
 'lemma_id': '25'}

In [28]:
data = pd.concat([pd.DataFrame.from_dict({"word": str(token), **token.feature._asdict()}, orient="index").T 
                  for token in tokens]).reset_index(drop=True)
data

,word,pos1,pos2,pos3,pos4,cType,cForm,lForm,lemma,orth,pron,orthBase,pronBase,goshu,iType,iForm,fType,fForm,iConType,fConType,type,kana,kanaBase,form,formBase,aType,aConType,aModType,lid,lemma_id
0,５,名詞,数詞,*,*,*,*,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,．,補助記号,句点,*,*,*,*,*,．,．,*,．,*,記号,*,*,*,*,*,*,補助,*,*,*,*,*,*,*,14027396882944,51
2,家,名詞,普通名詞,一般,*,*,*,イエ,家,家,イエ,家,イエ,和,*,*,*,*,*,*,体,イエ,イエ,イエ,イエ,2,C3,*,434315716600320,1580
3,に,助詞,格助詞,*,*,*,*,ニ,に,に,ニ,に,ニ,和,*,*,*,*,*,*,格助,ニ,ニ,ニ,ニ,*,名詞%F1,*,7745518285496832,28178
4,帰る,動詞,一般,*,*,五段-ラ行,連体形-一般,カエル,返る,帰る,カエル,帰る,カエル,和,*,*,*,*,*,*,用,カエル,カエル,カエル,カエル,1,C1,*,1643503629247169,5979
5,とき,名詞,普通名詞,副詞可能,*,*,*,トキ,時,とき,トキ,とき,トキ,和,ト濁,基本形,*,*,*,*,体,トキ,トキ,トキ,トキ,2,C3,*,7210331000676864,26231
6,大きな,連体詞,*,*,*,*,*,オオキナ,大きな,大きな,オーキナ,大きな,オーキナ,和,*,*,*,*,*,*,相,オオキナ,オオキナ,オオキナ,オオキナ,1,*,*,1254551424475648,4564
7,地震,名詞,普通名詞,一般,*,*,*,ジシン,地震,地震,ジシン,地震,ジシン,漢,*,*,*,*,*,*,体,ジシン,ジシン,ジシン,ジシン,0,C2,*,4908503407927808,17857
8,が,助詞,格助詞,*,*,*,*,ガ,が,が,ガ,が,ガ,和,*,*,*,*,*,*,格助,ガ,ガ,ガ,ガ,*,"動詞%F2@0,名詞%F1",*,2168520431510016,7889
9,起こる,動詞,一般,*,*,五段-ラ行,終止形-一般,オコル,起こる,起こる,オコル,起こる,オコル,和,*,*,*,*,*,*,用,オコル,オコル,オコル,オコル,2,C1,*,1341137965163179,4879


In [29]:
attr = ["orth", "lemma", "pos1", "cType", "kana"]

In [30]:
data[attr]

,orth,lemma,pos1,cType,kana
0,None,None,名詞,*,None
1,．,．,補助記号,*,*
2,家,家,名詞,*,イエ
3,に,に,助詞,*,ニ
4,帰る,返る,動詞,五段-ラ行,カエル
5,とき,時,名詞,*,トキ
6,大きな,大きな,連体詞,*,オオキナ
7,地震,地震,名詞,*,ジシン
8,が,が,助詞,*,ガ
9,起こる,起こる,動詞,五段-ラ行,オコル


In [31]:
from pykakasi import kakasi

In [32]:
kks = kakasi()
kks.setMode("K", "H")
conv = kks.getConverter()
conv.do("カタカナ")

'かたかな'

In [33]:
def furiganize(text):
    kks = kakasi()
    kks.setMode("K", "H")
    conv = kks.getConverter()
    
    tagger = fugashi.Tagger()
    
    def convert(toktext):
        from extra import stop_words
        
        token, text = toktext
        feature = token.feature
        
        if feature.orthBase is None:
            out = str(token)
        elif (feature.orthBase, conv.do(feature.kanaBase)) in stop_words:
            out = feature.orth        
        elif feature.kana == "*" or feature.kana == feature.orth:
            out = feature.orth
        else:
            kana = conv.do(feature.kana)
            
            if kana == feature.orth:
                out = feature.orth
            else:
                k = -1
                
                while feature.orth[k] == kana[k]:
                    k -= 1
                    
                k += 1
                
                if k == 0:
                    out = rf"\f{{{feature.orth}}}{{{kana}}}"
                else:                   
                    out = rf"\f{{{feature.orth[:k]}}}{{{kana[:k]}}}{feature.orth[k:]}"
        
        return token, out
            
    def box(toktext):
        token, text = toktext
        return token, rf"\p{{{text}}}"
    
    def wrap(token):
        return token, ""
    
    def finish(toktext):
        token, text = toktext
        return text
    
    toktexts = [convert(wrap(token)) for token in tagger(text)]
    
    prev_token, prev_text = None, None
    box = False
    last = len(toktexts) - 1
    
    def should_close_box(token):
        return (token.feature.pos1 in ["名詞", "代名詞", "形状詞", "連体詞", "副詞", "接続詞", "感動詞", "動詞", "形容詞", "接頭辞"]
                or should_keep_box(token))
    
    def should_keep_box(token):
        return token.feature.pos1 == "補助記号" and token.feature.orth in ["「", "（", "＜", "(", "["]
    
    keep_box = False
    
    for k, (token, text) in enumerate(toktexts):
        if k == 0:
            text = r"\p{" + text
        elif should_close_box(token) and not keep_box:
            text = r"}\p{" + text
            
        keep_box = should_keep_box(token)
            
        if k == last:
            text = text + r"}"
            
        toktexts[k] = token, text
        prev_token, prev_text = token, text
        
    return str.join("", [finish(toktext) for toktext in toktexts])

In [34]:
print(*[furiganize(t) for t in text.split("\n")], sep="\n")

\p{５．}\p{\f{家}{いえ}に}\p{\f{帰}{かえ}る}\p{とき}
\p{大きな}\p{\f{地震}{じしん}が}\p{\f{起}{お}こると、}\p{\f{電車}{でんしゃ}や}\p{バスなどが}\p{\f{止}{と}まる}\p{ことが}\p{あります。}\p{\f{急}{いそ}いで}\p{\f{家}{いえ}に}\p{\f{帰}{かえ}ろうと}\p{しないで、}\p{\f{会社}{かいしゃ}や}\p{\f{学校}{がっこう}など}\p{\f{安全}{あんぜん}な}\p{\f{場所}{ばしょ}で}\p{しばらく}\p{\f{待}{ま}って}\p{ください。}\p{\f{大勢}{おおぜい}の}\p{\f{人}{ひと}が}\p{\f{同}{おな}じ}\p{\f{時間}{じかん}に}\p{\f{帰}{かえ}ろうと}\p{すると、}\p{\f{道}{みち}や}\p{\f{駅}{えき}などが}\p{\f{混}{こ}んで}\p{\f{危険}{きけん}だからです。}\p{テレビや}\p{インターネットなどで}\p{\f{調}{しら}べて、}\p{\f{安全}{あんぜん}だと}\p{わかってから}\p{\f{帰}{かえ}りましょう。}

\p{６．}\p{\f{家族}{かぞく}や}\p{\f{友達}{ともだち}に}\p{\f{連絡}{れんらく}}\p{する}\p{とき}

\p{\f{地震}{じしん}の}\p{あとは}\p{\f{大勢}{おおぜい}の}\p{\f{人}{ひと}が}\p{\f{電話}{でんわ}を}\p{\f{使}{つか}う}\p{ため、}\p{\f{家族}{かぞく}や}\p{\f{友達}{ともだち}に}\p{\f{連絡}{れんらく}}\p{しにくく}\p{なります。}\p{\f{電話}{でんわ}}\p{\f{番号}{ばんごう}}\p{１７１の}\p{「\f{災害}{さいがい}\f{用}{よう}}\p{\f{伝言}{でんごん}}\p{ダイヤル」を}\p{\f{使}{つか}うと、}\p{\f{会話}{かいわ}は}\p{できなくても}\p{メッセージを}\p{\f{録音}{ろくおん}}\p{したり、}\p{\f{聞}{き}いたり}\p{する}\p{ことが}\p{できます。}\p{\f{携帯}{けいたい}}\p{\f{電話}{でんわ}}\p{\f{会社}{かいし

In [35]:
text2 = r"""
どうして記事は急いで家に帰ることをお勧めしないのですか？
どこで情報を集めることができますか？
どの様に家族や友達と連絡をとることができますか？
どうして避難所にいる時健康に気を付けなければなりませんか？
エコノミークラス症候群はどういう病気なんですか？
""".strip()

text3= r"""
_____うちまで武君を連れてくれ。
ファイアーウォールがインターネットの_____を保障する。
花子ちゃんは警察に____します。
鈴木先生に何の____？
酷い目にあいたくないなら、武君と____に関わらない方がいいです。
貴重な花瓶が_____武君は困っている。
親と_______住むのがとても不便です。
""".strip()

In [37]:
print(*[r"\item " + furiganize(row) for row in text2.split("\n")], sep="\n")

\item \p{どう}\p{して}\p{\f{記事}{きじ}は}\p{\f{急}{いそ}いで}\p{\f{家}{いえ}に}\p{\f{帰}{かえ}る}\p{ことを}\p{お}\p{\f{勧}{すす}め}\p{しないのですか？}
\item \p{どこで}\p{\f{情報}{じょうほう}を}\p{\f{集}{あつ}める}\p{ことが}\p{できますか？}
\item \p{どの}\p{\f{様}{よう}に}\p{\f{家族}{かぞく}や}\p{\f{友達}{ともだち}と}\p{\f{連絡}{れんらく}を}\p{とる}\p{ことが}\p{できますか？}
\item \p{どう}\p{して}\p{\f{避難}{ひなん}\f{所}{しょ}に}\p{いる}\p{時}\p{\f{健康}{けんこう}に}\p{\f{気}{き}を}\p{\f{付}{つ}けなければ}\p{なりませんか？}
\item \p{エコノミー}\p{クラス}\p{\f{症候}{しょうこう}}\p{\f{群}{ぐん}は}\p{どう}\p{いう}\p{\f{病気}{びょうき}なんですか？}


In [16]:
pd.concat([pd.DataFrame.from_dict({"word": str(token), **token.feature._asdict()}, orient="index").T
           for token in tagger(text3)]).reset_index(drop=True)[attr]

,orth,lemma,pos1,cType,kana
0,危険,危険,名詞,*,キケン
1,な,だ,助動詞,助動詞-ダ,ナ
2,もの,物,名詞,*,モノ
3,が,が,助詞,*,ガ
4,起こる,起こる,動詞,五段-ラ行,オコル
5,と,と,助詞,*,ト
6,、,、,補助記号,*,*
7,すぐ,直ぐ,副詞,*,スグ
8,に,に,助詞,*,ニ
9,助け,助け,名詞,*,タスケ


In [17]:
from extra import stop_words
stop_words

[('車', 'くるま'),
 ('上', 'うえ'),
 ('下', 'した'),
 ('入る', 'はいる'),
 ('火', 'ひ'),
 ('外', 'そと'),
 ('出る', 'でる'),
 ('出す', 'だす'),
 ('大きい', 'おおきい'),
 ('大きな', 'おおきな'),
 ('山', 'やま'),
 ('中', 'なか'),
 ('何', 'なに'),
 ('時', 'とき'),
 ('見る', 'みる'),
 ('今', 'いま'),
 ('前', 'まえ'),
 ('話す', 'はなす'),
 ('一', 'いち'),
 ('二', 'に'),
 ('三', 'さん'),
 ('四', 'よん'),
 ('五', 'ご'),
 ('六', 'ろく'),
 ('七', 'しち'),
 ('八', 'はち'),
 ('九', 'きゅう'),
 ('十', 'じゅう'),
 ('百', 'ひゃく'),
 ('千', 'せん'),
 ('万', 'まん')]

In [18]:
tagger(text)[0].feature.pos1

'名詞'

In [19]:
import jamdict
jmd = jamdict.Jamdict()

In [20]:
result = jmd.lookup("や")

In [21]:
result.entries[5].senses[0].pos[0]

'particle'

In [22]:
tokens[8].feature.pos1

'助詞'

In [23]:
[entry for entry in result.entries if any(["particle" in sense.pos for sense in entry.senses])]

[や : 1. such things as (non-exhaustive list related to a specific time and place)/and ... and 2. the minute (that) .../no sooner than .../as soon as 3. punctuational exclamation in haiku, renga, etc. 4. interjection expressing surprise 5. hi 6. plain copula 7. yes/what? 8. indicates a question]

In [24]:
pd.concat([pd.DataFrame.from_dict({"word": str(token), **token.feature._asdict()}, orient="index").T
           for token in tagger("健二郎さんは今夜お酒ばかりを飲んでいます。")]).reset_index(drop=True)[["orth", "lemma", "pos1", "pos2"]]

,orth,lemma,pos1,pos2
0,健二郎,ケンジロウ,名詞,固有名詞
1,さん,さん,接尾辞,名詞的
2,は,は,助詞,係助詞
3,今夜,今夜,名詞,普通名詞
4,お,御,接頭辞,*
5,酒,酒,名詞,普通名詞
6,ばかり,ばかり,助詞,副助詞
7,を,を,助詞,格助詞
8,飲ん,飲む,動詞,一般
9,で,て,助詞,接続助詞


In [35]:
pd.concat([pd.DataFrame.from_dict({"word": str(token), **token.feature._asdict()}, orient="index").T
           for token in tagger("誰かがいる。")]).reset_index(drop=True)[["orth", "lemma", "pos1", "pos2"]]

,orth,lemma,pos1,pos2
0,誰,誰,代名詞,*
1,か,か,助詞,副助詞
2,が,が,助詞,格助詞
3,いる,居る,動詞,非自立可能
4,。,。,補助記号,句点
